In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [43.3 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [456 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [833 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Show DataFrame
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('homes')


In [11]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

spark.sql("SELECT date_built, ROUND(AVG(price),2) AS RoundValue FROM homes WHERE bedrooms=4 GROUP BY date_built ORDER BY date_built").show()

+----------+----------+
|date_built|RoundValue|
+----------+----------+
|      2010| 296800.75|
|      2011|  302141.9|
|      2012| 298233.42|
|      2013| 299999.39|
|      2014| 299073.89|
|      2015| 307908.86|
|      2016| 296050.24|
|      2017| 296576.69|
+----------+----------+



In [17]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("SELECT date_built, ROUND(AVG(price),2) AS RoundValue FROM homes WHERE bedrooms=3 AND bathrooms=3 GROUP BY date_built ORDER BY date_built").show()

+----------+----------+
|date_built|RoundValue|
+----------+----------+
|      2010| 292859.62|
|      2011| 291117.47|
|      2012| 293683.19|
|      2013| 295962.27|
|      2014| 290852.27|
|      2015|  288770.3|
|      2016| 290555.07|
|      2017| 292676.79|
+----------+----------+



In [18]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("SELECT date_built, ROUND(AVG(price),2) AS RoundValue FROM homes WHERE bedrooms=3 AND bathrooms=3 AND floors=2 AND sqft_living >2000 GROUP BY date_built ORDER BY date_built").show()

+----------+----------+
|date_built|RoundValue|
+----------+----------+
|      2010| 285010.22|
|      2011| 276553.81|
|      2012| 307539.97|
|      2013| 303676.79|
|      2014| 297619.46|
|      2015| 297609.97|
|      2016|  293965.1|
|      2017| 280527.62|
+----------+----------+



In [38]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) AS RoundValue FROM homes WHERE price>=350000 GROUP BY 1").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|RoundValue|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- 0.29622507095336914 seconds ---


In [28]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table homes")

DataFrame[]

In [29]:
# 8. Check if the table is cached.
spark.catalog.isCached('homes')

True

In [39]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) AS RoundValue FROM homes WHERE price>=350000 GROUP BY 1").show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view|RoundValue|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- 0.6286911964416504 seconds ---


In [40]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.parquet('parquet_homes', mode='overwrite')

In [41]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('parquet_homes')

In [42]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_homes')

In [43]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) AS RoundValue FROM p_homes WHERE price>=350000 GROUP BY 1").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|RoundValue|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- 1.3162250518798828 seconds ---


In [44]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table homes")

DataFrame[]

In [46]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("homes")

False